# Bài Tập 3: Mạng Nơ Ron Một Lớp Ẩn Cho Nhận Dạng Chữ Số MNIST

Giả sử ta có một mạng nơ ron gồm:
- **Input layer:** 784 đặc trưng (mỗi ảnh được phẳng thành vector 784 chiều).
- **Lớp ẩn:** 30 nơ ron, sử dụng hàm kích hoạt sigmoid.
- **Lớp output:** 10 nơ ron (cho 10 lớp chữ số 0-9), sử dụng hàm kích hoạt sigmoid.
- **Hàm Loss:** Cross-Entropy (áp dụng cho từng đầu ra, tính trung bình qua $m$ mẫu).

Trong các phần dưới đây, ta sẽ trình bày:
1. Phương trình forward (tính lan truyền).
2. Tính đạo hàm (Back-Propagation) dưới dạng ma trận.
3. Các bước của thuật toán Gradient Descent.

---

## Câu 1: Phương Trình Forward

Giả sử ta có tập dữ liệu gồm $m$ mẫu:
- **Input:** $ X \in \mathbb{R}^{784 \times m} $

**Lớp ẩn (Hidden Layer):**
- Trọng số: $ W^{[1]} \in \mathbb{R}^{30 \times 784} $
- Bias: $ b^{[1]} \in \mathbb{R}^{30 \times 1} $
- Tính tổng tuyến tính:
  $
  Z^{[1]} = W^{[1]} X + b^{[1]} \quad \in \mathbb{R}^{30 \times m}
  $
- Áp dụng hàm kích hoạt sigmoid:
  $
  A^{[1]} = \sigma\bigl(Z^{[1]}\bigr) = \frac{1}{1 + e^{-{Z^{[1]}}}} \quad \in \mathbb{R}^{30 \times m}
  $

**Lớp output (Output Layer):**
- Trọng số: $ W^{[2]} \in \mathbb{R}^{10 \times 30} $
- Bias: $ b^{[2]} \in \mathbb{R}^{10 \times 1} $
- Tính tổng tuyến tính:
  $
  Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]} \quad \in \mathbb{R}^{10 \times m}
  $
- Áp dụng hàm kích hoạt sigmoid:
  $
  A^{[2]} = \sigma\bigl(Z^{[2]}\bigr) = \frac{1}{1 + e^{-{Z^{[2]}}}} \quad \in \mathbb{R}^{10 \times m}
  $

**Hàm Loss (Cross-Entropy):**  
Giả sử nhãn đúng được mã hóa one-hot, ta có:
- $ Y \in \mathbb{R}^{10 \times m} $ với $ m $ là số mẫu.
- $ A^{[2]} \in \mathbb{R}^{10 \times m} $ là đầu ra của lớp output (sau khi áp dụng hàm sigmoid).

Hàm loss cho mỗi mẫu $ i $ được tính theo công thức:
$
L^{(i)} = -\sum_{j=1}^{10} \Bigl[ Y_{j}^{(i)} \log\Bigl( A_{j}^{[2](i)} \Bigr) + \Bigl( 1 - Y_{j}^{(i)} \Bigr) \log\Bigl( 1 - A_{j}^{[2](i)} \Bigr) \Bigr]
$

Hàm loss trung bình trên toàn bộ tập mẫu là:
$
J = \frac{1}{m} \sum_{i=1}^{m} L^{(i)}
$

---

## Câu 2: Tính Đạo Hàm 

### 1. Đạo hàm Ở Lớp Output

- **Đầu ra của lớp output:**
  $
  A^{[2]} = \sigma\bigl(Z^{[2]}\bigr)
  $
- Với hàm sigmoid kết hợp với Cross-Entropy, ta có đạo hàm theo từng mẫu:
  $
  \frac{\partial L^{(i)}}{\partial Z^{[2](i)}} = A^{[2](i)} - Y^{(i)}
  $
- Dạng ma trận cho toàn bộ tập mẫu:
  $
  dZ^{[2]} = A^{[2]} - Y \quad \in \mathbb{R}^{10 \times m}
  $
- Tính đạo hàm theo $W^{[2]}$ và $b^{[2]}$:
  $
  dW^{[2]} = \frac{1}{m} \, dZ^{[2]} (A^{[1]})^T \quad \in \mathbb{R}^{10 \times 30}
  $
  $
  db^{[2]} = \frac{1}{m} \sum_{i=1}^{m} dZ^{[2](i)} \quad \in \mathbb{R}^{10 \times 1}
  $

### 2. Đạo hàm Ở Lớp Ẩn

**Bước 1: Lan Truyền Lỗi Về Lớp Ẩn**

- Lỗi từ lớp output được lan truyền về lớp ẩn thông qua trọng số:
  $
  dA^{[1]} = (W^{[2]})^T dZ^{[2]} \quad \in \mathbb{R}^{30 \times m}
  $

**Bước 2: Tính Đạo Hàm của Hàm Kích Hoạt Sigmoid**

- Hàm kích hoạt của lớp ẩn:
  $
  A^{[1]} = \sigma\bigl(Z^{[1]}\bigr)
  $
- Đạo hàm của sigmoid theo $Z^{[1]}$ (theo từng phần tử):
  $
  \frac{\partial A^{[1]}}{\partial Z^{[1]}} = A^{[1]} \circ (1-A^{[1]})
  $
  (Ở đây $\circ$ biểu thị phép nhân phần tử.)

**Bước 3: Tính $dZ^{[1]}$ cho Lớp Ẩn**

- Áp dụng quy tắc chuỗi:
  $
  dZ^{[1]} = dA^{[1]} \circ \Bigl(A^{[1]} \circ (1-A^{[1]})\Bigr)\quad \in \mathbb{R}^{30 \times m}
  $
- 
  Hay, với mỗi phần tử:
  $
  [dZ^{[1]}]_{ij} = [dA^{[1]}]_{ij} \times [A^{[1]}]_{ij} \times \bigl(1-[A^{[1]}]_{ij}\bigr)
  $

**Bước 4: Tính Đạo Hàm Theo $W^{[1]}$ và $b^{[1]}$**

- Đạo hàm theo $W^{[1]}$:
  $
  dW^{[1]} = \frac{1}{m} \, dZ^{[1]} \, X^T \quad \in \mathbb{R}^{30 \times 784}
  $
- Đạo hàm theo $b^{[1]}$:
  $
  db^{[1]} = \frac{1}{m} \sum_{i=1}^{m} dZ^{[1](i)} \quad \in \mathbb{R}^{30 \times 1}
  $
  (Tổng các cột của $dZ^{[1]}$.)

---

## Câu 3: Mô Tả Các Bước Của Thuật Toán Gradient Descent

Quy trình huấn luyện bằng Gradient Descent bao gồm các bước sau:

1. **Khởi tạo tham số:**
   - Khởi tạo $ W^{[1]} \in \mathbb{R}^{30 \times 784} $ và $ b^{[1]} \in \mathbb{R}^{30 \times 1} $ (có thể khởi tạo ngẫu nhiên).
   - Khởi tạo $ W^{[2]} \in \mathbb{R}^{10 \times 30} $ và $ b^{[2]} \in \mathbb{R}^{10 \times 1} $.

2. **Forward Propagation:**
   - Tính $ Z^{[1]} = W^{[1]} X + b^{[1]} $.
   - Tính $ A^{[1]} = \sigma\bigl(Z^{[1]}\bigr) $.
   - Tính $ Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]} $.
   - Tính $ A^{[2]} = \sigma\bigl(Z^{[2]}\bigr) $.

3. **Tính Loss:**
   - Tính hàm loss Cross-Entropy trên toàn bộ tập mẫu:
$
J = -\frac{1}{m} \sum_{i=1}^{m} \sum_{j=1}^{10} \Bigl[ Y_{j}^{(i)} \log\Bigl(A_{j}^{[2](i)}\Bigr) + \Bigl(1-Y_{j}^{(i)}\Bigr) \log\Bigl(1-A_{j}^{[2](i)}\Bigr) \Bigr]
$

4. **Backward Propagation:**
   - **Lớp Output:**
     - Tính $ dZ^{[2]} = A^{[2]} - Y $.
     - Tính $ dW^{[2]} = \frac{1}{m} \, dZ^{[2]} (A^{[1]})^T $.
     - Tính $ db^{[2]} = \frac{1}{m} \sum dZ^{[2]} $.
   - **Lớp Ẩn:**
     - Lan truyền lỗi: $ dA^{[1]} = (W^{[2]})^T dZ^{[2]} $.
     - Tính $ dZ^{[1]} = dA^{[1]} \circ \Bigl(A^{[1]} \circ (1-A^{[1]})\Bigr) $.
     - Tính $ dW^{[1]} = \frac{1}{m} \, dZ^{[1]} \, X^T $.
     - Tính $ db^{[1]} = \frac{1}{m} \sum dZ^{[1]} $.

5. **Cập nhật Tham Số:**
   - Cập nhật các tham số theo công thức:
     $
     W^{[l]} := W^{[l]} - \alpha \, dW^{[l]} \quad \text{với } l=1,2
     $
     $
     b^{[l]} := b^{[l]} - \alpha \, db^{[l]} \quad \text{với } l=1,2
     $
   - Ở đây $\alpha$ là learning rate.

6. **Lặp Lại:**
   - Thực hiện các bước 2–5 trong số vòng lặp (iterations) cho đến khi hàm loss hội tụ hoặc đạt số vòng lặp tối đa.




In [1]:
import numpy as np

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def forward_propagation(X, W1, b1, W2, b2):
    # Lớp ẩn
    Z1 = np.dot(W1, X) + b1        # (30, m)
    A1 = sigmoid(Z1)               # (30, m)
    # Lớp output
    Z2 = np.dot(W2, A1) + b2       # (10, m)
    A2 = sigmoid(Z2)               # (10, m)
    cache = {"Z1": Z1, "A1": A1, "Z2": Z2, "A2": A2}
    return A2, cache

def compute_loss(A2, Y):
    m = Y.shape[1]
    loss = - (1/m) * np.sum(Y * np.log(A2 + 1e-8) + (1 - Y) * np.log(1 - A2 + 1e-8))
    return loss

def backward_propagation(X, Y, cache, W2):
    m = X.shape[1]
    A1 = cache["A1"]
    A2 = cache["A2"]
    # Lớp output
    dZ2 = A2 - Y                              # (10, m)
    dW2 = (1/m) * np.dot(dZ2, A1.T)             # (10, 30)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)  # (10, 1)
    # Lớp ẩn
    dA1 = np.dot(W2.T, dZ2)                     # (30, m)
    dZ1 = dA1 * (A1 * (1 - A1))                 # (30, m)
    dW1 = (1/m) * np.dot(dZ1, X.T)              # (30, 784)
    db1 = (1/m) * np.sum(dZ1, axis=1, keepdims=True)  # (30, 1)
    
    grads = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}
    return grads

def update_parameters(W1, b1, W2, b2, grads, learning_rate):
    W1 = W1 - learning_rate * grads["dW1"]
    b1 = b1 - learning_rate * grads["db1"]
    W2 = W2 - learning_rate * grads["dW2"]
    b2 = b2 - learning_rate * grads["db2"]
    return W1, b1, W2, b2

def model(X, Y, num_iterations=10000, learning_rate=0.1, print_cost=True):
    np.random.seed(1)
    m = X.shape[1]
    # Khởi tạo tham số
    W1 = np.random.randn(30, 784) * 0.01
    b1 = np.zeros((30, 1))
    W2 = np.random.randn(10, 30) * 0.01
    b2 = np.zeros((10, 1))
    
    for i in range(num_iterations):
        A2, cache = forward_propagation(X, W1, b1, W2, b2)
        cost = compute_loss(A2, Y)
        grads = backward_propagation(X, Y, cache, W2)
        W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, grads, learning_rate)
        if print_cost and i % 1000 == 0:
            print(f"Iteration {i}, cost: {cost:.6f}")
    
    parameters = {"W1": W1, "b1": b1, "W2": W2, "b2": b2}
    return parameters

# Giả sử X là ma trận dữ liệu có kích thước (784, m)
# Và Y là ma trận one-hot có kích thước (10, m)
# Sau đó gọi:
# parameters = model(X, Y, num_iterations=10000, learning_rate=0.1)